## Setting up the notebook

In [ ]:
# installing packages
!pip install pip==24.0
!pip install numpy==1.23.5
!pip install tensorboardX
!pip install sentencepiece

  Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl.metadata (5.8 kB)
Using cached tensorboardX-2.6.2.2-py2.py3-none-any.whl (101 kB)


In [ ]:
# importing packages
import numpy
import os
import tensorboardX
import sentencepiece as spm

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
source_code = 'eng'
target_code = 'nso'

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing optok
#!git clone https://github.com/tatHi/optok4at.git
%cd optok4at/machineTranslation/optok
!pip install --editable .

/content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/optok
Obtaining file:///content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/optok
  Preparing metadata (setup.py) ... done
  Running setup.py develop for optok_nmt


In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# installing fairseq
%cd optok4at/machineTranslation/fairseq
!pip install --editable .

/content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/fairseq
Obtaining file:///content/drive/MyDrive/Research/eng-to-nso/optok4at/machineTranslation/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.2 MB/s eta 0:00:00
  Building editable for fairseq (pyproject.toml) ... done
  Created wheel for fairseq: filename=fairseq-0.9.0-0.editable-cp310-cp310-linux_x86_64.whl size=7879 sha256=0a6afd829054434aacfc88f69ecaa4fd85c4f72ea7a347d7af8d73bc55ce0e95
  Stored in directory: /tmp/pip-ephem-wheel-cache-68lv6ipe/wheels/df/7c/ad/dad45b2aba84494676571009da0d5e5d4dc11b5a3eacbeb9fb
Successfully built fairseq


In [ ]:
# installing youtokentome
!pip install youtokentome
import youtokentome as yttm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for youtokentome: filename=youtokentome-1.0.6-cp310-cp310-linux_x86_64.whl size=1951577 sha256=376863cc952b27dca1a85e56d7dbf21cf073262d9e2f83788f4e05d506a0d234
  Stored in directory: /root/.cache/pip/wheels/df/85/f8/301d2ba45f43f30bed2fe413efa760bc726b8b660ed9c2900c
Successfully built youtokentome


In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}')

# install multigram
#!git clone https://github.com/tatHi/multigram
%cd multigram
!pip install --editable .

/content/drive/MyDrive/Research/eng-to-nso/multigram
Obtaining file:///content/drive/MyDrive/Research/eng-to-nso/multigram
  Preparing metadata (setup.py) ... done
  Running setup.py develop for multigram



## Preparing the parallel corpus

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok')

In [ ]:
# prepare dictionary to be used by our nmt model
!fairseq-preprocess -s eng -t nso \
--trainpref /content/drive/MyDrive/Research/eng-to-nso/ulm/data/train.ULM \
--validpref /content/drive/MyDrive/Research/eng-to-nso/ulm/data/val.ULM \
--testpref /content/drive/MyDrive/Research/eng-to-nso/ulm/data/test.ULM \
--joined-dictionary \
--srcdict /content/drive/MyDrive/Research/eng-to-nso/ulm/data-bin/dict.eng.txt \
--destdir data-bin \
--bpe 'sentencepiece' \
--workers 2

2024-11-01 09:14:21 | INFO | fairseq_cli.preprocess | Namespace(no_progress_bar=False, log_interval=100, log_format=None, tensorboard_logdir='', seed=None, cpu=False, tpu=False, bf16=False, fp16=False, memory_efficient_bf16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, checkpoint_suffix='', quantization_config_path=None, profile=False, criterion='cross_entropy', tokenizer=None, bpe='sentencepiece', optimizer=None, lr_scheduler='fixed', scoring='bleu', task='translation', source_lang='eng', target_lang='nso', trainpref='/content/drive/MyDrive/Research/eng-to-nso/ulm/data/train.ULM', validpref='/content/drive/MyDrive/Research/eng-to-nso/ulm/data/val.ULM', testpref='/content/drive/MyDrive/Research/eng-to-nso/ulm/data/test.ULM', align_suffix=None, destdir='data-bin', 

## Training Tokenizer

In [ ]:
# change working directory
os.chdir(f'/content/drive/MyDrive/Research/eng-to-{target_code}/target-tok')

In [ ]:
!fairseq-train \
--arch transformer_optok \
--activation-fn relu \
--share-decoder-input-output-embed \
--share-all-embeddings \
--encoder-layers 3 \
--encoder-attention-heads 4 \
--encoder-embed-dim 256 \
--encoder-ffn-embed-dim 1024 \
--decoder-layers 3 \
--decoder-attention-heads 4 \
--decoder-embed-dim 256 \
--decoder-ffn-embed-dim 1024 \
--dropout 0.25 \
--seed 2024 \
--optimizer 'adam' \
--adam-betas '(0.9, 0.999)' \
--lr-scheduler 'inverse_sqrt' \
--patience 5 \
--warmup-updates 1000 \
--criterion optok_pass_through \
--lr 0.0003 \
--weight-decay 0.0 \
--max-tokens 4096 \
--max-tokens-valid 3600 \
--required-batch-size-multiple 1 \
--update-freq 1 \
--max-epoch 30 \
--validate-interval 10 \
--save-interval 10 \
--best-checkpoint-metric 'loss' \
--log-interval 100 \
--curriculum 0 \
--save-dir checkpoints-target-tok \
--ddp-backend=no_c10d \
--optok-sp-src-path /content/drive/MyDrive/Research/eng-to-nso/ulm/data/joint.model \
--optok-sp-tgt-path /content/drive/MyDrive/Research/eng-to-nso/ulm/data/joint.model \
--optok-m 3 \
--sp-alpha 0.2 \
--optok-normal-enc \
--optok-lmemb-dec /content/drive/MyDrive/Research/eng-to-nso/target-tok/checkpoints-target-tok/cache/joint.model.lmemb.pt \
--optok-nlm-dec /content/drive/MyDrive/Research/eng-to-nso/target-tok/checkpoints-target-tok/cache/joint.model.nlm.pt \
/content/drive/MyDrive/Research/eng-to-nso/target-tok/data-bin

2024-11-06 04:13:41 | INFO | fairseq_cli.train | Namespace(no_progress_bar=False, log_interval=100, log_format=None, tensorboard_logdir='', seed=2024, cpu=False, tpu=False, bf16=False, fp16=False, memory_efficient_bf16=False, memory_efficient_fp16=False, fp16_no_flatten_grads=False, fp16_init_scale=128, fp16_scale_window=None, fp16_scale_tolerance=0.0, min_loss_scale=0.0001, threshold_loss_scale=None, user_dir=None, empty_cache_freq=0, all_gather_list_size=16384, model_parallel_size=1, checkpoint_suffix='', quantization_config_path=None, profile=False, criterion='optok_pass_through', tokenizer=None, bpe=None, optimizer='adam', lr_scheduler='inverse_sqrt', scoring='bleu', task='translation', num_workers=1, skip_invalid_size_inputs_valid_test=False, max_tokens=4096, max_sentences=None, required_batch_size_multiple=1, dataset_impl=None, data_buffer_size=10, train_subset='train', valid_subset='valid', validate_interval=10, validate_interval_updates=0, validate_after_updates=0, fixed_valida